# LeNet


![img](./img/LeNet.jpg)

In [1]:
import torch
from torch import nn

/opt/homebrew/Caskroom/miniforge/base/envs/Atari/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Reshape(nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 28, 28)
    
net = torch.nn.Sequential(
        Reshape(), 
        nn.Conv2d(1, 6, kernel_size=5, padding=2), # 6 * 28 * 28
        nn.Sigmoid(), 
        nn.AvgPool2d(2, stride=2),# 6 * 14 * 14
        nn.Conv2d(6, 16, kernel_size=5),# 16 * 10 * 10
        nn.Sigmoid(),
        nn.AvgPool2d(2, stride=2),# 16 * 5 * 5
        nn.Flatten(),
        nn.Linear(16*5*5, 120),nn.Sigmoid(),
        nn.Linear(120, 84),nn.Sigmoid(),
        nn.Linear(84, 10)
        )

In [4]:
X = torch.rand(size=(1,1,28,28), dtype=torch.float32)

for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t\t', X.shape)

Reshape output shape: 		 torch.Size([1, 1, 28, 28])
Conv2d output shape: 		 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 		 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 		 torch.Size([1, 6, 14, 14])
Conv2d output shape: 		 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 		 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 		 torch.Size([1, 16, 5, 5])
Flatten output shape: 		 torch.Size([1, 400])
Linear output shape: 		 torch.Size([1, 120])
Sigmoid output shape: 		 torch.Size([1, 120])
Linear output shape: 		 torch.Size([1, 84])
Sigmoid output shape: 		 torch.Size([1, 84])
Linear output shape: 		 torch.Size([1, 10])


## train

In [10]:
import torchvision
from torchvision import transforms
from torch.utils import data

batch_size = 256
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    # 如果有resize，先做resize，然后再转换到tensor
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root='./data/', train=True, transform=trans, download=False)
    mnist_test = torchvision.datasets.FashionMNIST("./data", train=False, transform=trans, download=False)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=8), 
           data.DataLoader(mnist_test, batch_size, shuffle=True, num_workers=8))
  
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [12]:
import tqdm as tqdm

In [23]:
def train(net, train_iter, test_iter, num_epochs, lr):
    def init_weights(m):
        if type(m)==nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
            
    net.apply(init_weights)
    print('begin training')
    
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        net.train()
        for i, (X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            y_hat = net(X)
            L = loss(y_hat, y)
            L.backward()
            optimizer.step()
            
        print(f'loss: {L.sum()}')

lr = 0.9
num_epochs = 10
train(net, train_iter, test_iter, num_epochs, lr)

## test accurancy

In [25]:
def accurancy(y, y_hat):
    if len(y_hat) > 1 and len(y_hat[0]) > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat == y
    return cmp.sum().item()

# 测试集上的效果
net.eval()

sum_accu = 0
for X, y in test_iter:
    y_hat = net(X)
    sum_accu += accurancy(y, y_hat)
    
sum_accu / 10000

# 在训练集上的效果
sum_accu = 0
for X, y in train_iter:
    y_hat = net(X)
    sum_accu += accurancy(y, y_hat)
    
sum_accu / 60000